### Imports

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import scipy as sp
from scipy import ndimage
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
import shapefile as sf
from scipy.interpolate import RegularGridInterpolator
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
from shapely.geometry import Point, Polygon

### Unpickle Smooth Subsampled Model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
#ifilename = './subsamp_smooth_z10.0m_nam_model_vp_vs_rho_Q_props.npz'
ifilename = './subsamp_smooth_z200m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d.shape:',gm3d.shape)

#free up some memory
del props 

### Confirm axes order

In [ ]:
gm3d.changeAxOrder({'X':2,'Y':1,'Z':0})
print(gm3d.shape)
gm3d.changeAxOrder({'X':0,'Y':1,'Z':2})
print(gm3d.shape)
gm3d.changeAxOrder({'X':1,'Y':2,'Z':0})
print(gm3d.shape)
gm3d.changeAxOrder({'X':0,'Y':1,'Z':2})
print(gm3d.shape)

### Setup all coordinates (also get bbox, etc...)

In [ ]:
mysf = sf.Reader('FieldShapeFile/Groningen_field')
print('mysf:',mysf)
print('mysf.shapes():',mysf.shapes())
s = mysf.shape(0)

sub_dxyz = 200

mybbox = s.bbox #this will be used for slicing (look further down)
print('mybbox:',mybbox)

#shrink and create y coordinates for slicing box
vl = np.array([0,0.87*(mybbox[3]-mybbox[1])])
dvl = ((0.87*(mybbox[3]-mybbox[1]))**2)**0.5
nvl = dvl//sub_dxyz + 1
y = np.arange(nvl)*sub_dxyz
print('nvl:',nvl)

#shrink and create x coordinates for slicing box
vb = np.array([0.85*(mybbox[2]-mybbox[0]),0])
dvb = ((0.85*(mybbox[2]-mybbox[0]))**2)**0.5
nvb = dvb//sub_dxyz + 1
x = np.arange(nvb)*sub_dxyz
print('nvb:',nvb)

#create set of xy coordinates for slicing box
xy = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
print('xy.shape:',xy.shape)

#setup rotation matrices
degree = 30
theta = degree*np.pi/180
rm = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])

#rotate coordinates
for i in range(len(xy[:,0])):
    xy[i,:] = rm.dot(xy[i,:])

#get translated coordinates
xshift = 12600
yshift = -2600
rxy = np.copy(xy)
rxy[:,0] += mybbox[0] + xshift
rxy[:,1] += mybbox[1] + yshift
print('rxy.shape:',rxy.shape)

### Slice Volume

In [ ]:
import time

#get sliced subsurface volume
start = time.time()
slice_props = gm3d.sliceVolumeValsFromCoordsXY(x,y,rxy,local=False)
end = time.time()

print('runtime:', end - start)

### Pickle the interpolated model

In [ ]:
import numpy as np
orrssslfqn = './rect_rot_subsamp_smooth_z' + str(dz) + 'm_nam_model_vp_vs_rho_Q_props.npz'
print(orrssslfqn)
np.savez_compressed(orrssslfqn,props=slice_props,xc=x,yc=y,rxyc=rxy)

### Unpickle the sliced volume if need be

In [ ]:
ifilename = './rect_rot_subsamp_smooth_z200.0m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
slice_props = data['props'] #4D ndarray
xc=data['xc']
yc=data['yc']
rxy=data['rxyc']

In [ ]:
print('slice_props.shape',slice_props.shape)

In [ ]:
sprops = np.copy(slice_props.reshape((4,31, 193, 146)),order='C')
print('sprops.shape:',sprops.shape)
print('gm3d.shape:',gm3d.shape)

In [ ]:
rdep_surf = sprops[0,10,:,:].copy()
print(rdep_surf.shape)
print('nxy:',rdep_surf.shape[0]*rdep_surf.shape[1])
print('nrxy:', rxy.shape)

# get new min max to normalize surface
vp_min = np.min(rdep_surf)
vp_max = np.max(rdep_surf)
surf_norm = Normalize(vp_min,vp_max)

xy = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])
print('xy.shape:',xy.shape)
print('xy:',xy)

fig, ax = plt.subplots(1,figsize=(6,6))
ax.scatter(rxy[:,0],rxy[:,1],s=1,c=rdep_surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
plt.show()

In [ ]:
sub_surf = gm3d[0,10,:,:].copy()
print(sub_surf.shape)
print('nxy:',sub_surf.shape[0]*sub_surf.shape[1])
print('nrxy:', rxy.shape)

# get new min max to normalize surface
vp_min = np.min(rdep_surf)
vp_max = np.max(rdep_surf)
surf_norm = Normalize(vp_min,vp_max)

sxc = 

xy = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])
print('xy.shape:',xy.shape)

fig, ax = plt.subplots(1,figsize=(6,6))
ax.scatter(xy[:,0],xy[:,1],s=1,c=sub_surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
plt.show()

In [ ]:
sprops = sprops.transpose(0,3,2,1).copy()
print('sprops.shape:',sprops.shape)

In [ ]:
print(np.isfortran(sprops))

In [ ]:
import
write_vtk_gridded_model_3d(fqpname,props,xdata,ydata,zdata):